In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./2.menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Item Number'].tolist()

总数量：1


['355350556206']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_proxy

if input_.loc[a, 'Market'] == 'US':
    url = f'''https://www.ebay.com/itm/{input_.loc[a, 'Item Number']}?_ul=US&_stpos=91710&orig_cvip=true'''
elif input_.loc[a, 'Market'] == 'DE':
    url = f'''https://www.ebay.de/itm/{input_.loc[a, 'Item Number']}?_ul=DE&_stpos=10115&orig_cvip=true'''
elif input_.loc[a, 'Market'] == 'UK':
    url = f'''https://www.ebay.co.uk/itm/{input_.loc[a, 'Item Number']}?_ul=GB&_stpos=BS81QU&orig_cvip=true'''
elif input_.loc[a, 'Market'] == 'AU':
    url = f'''https://www.ebay.com.au/itm/{input_.loc[a, 'Item Number']}?_ul=AU&_stpos=2019&orig_cvip=true'''

resp = requests.get(url,
                    proxies=get_proxy(),
                    timeout=(10, 10)).text

resp

'<!DOCTYPE html><html lang=en><head><link rel=dns-prefetch href=//thumbs.ebaystatic.com><link rel=dns-prefetch href=//vi.vipr.ebaydesc.com><link rel=dns-prefetch href=//p.ebaystatic.com><link rel=dns-prefetch href=//thumbs.ebaystatic.com><link rel=dns-prefetch href=//q.ebaystatic.com><link rel=dns-prefetch href=//pics.ebaystatic.com><link rel=dns-prefetch href=//srx.main.ebayrtm.com><link rel=dns-prefetch href=//reco.ebay.com><link rel="dns-prefetch" href="//ir.ebaystatic.com"><link rel="dns-prefetch" href="//secureir.ebaystatic.com"><link rel="dns-prefetch" href="//i.ebayimg.com"><link rel="dns-prefetch" href="//rover.ebay.com"><script>$ssgST=new Date().getTime();</script><style>\n    .font-marketsans body {\n        font-family: "Market Sans", Arial, sans-serif;\n    }\n</style><script>\n    (function() {\n        var useCustomFont = (\'fontDisplay\' in document.documentElement.style) ||\n                        (localStorage && localStorage.getItem(\'ebay-font\'));\n        if (useC

In [6]:
import json

json_ = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')

json_

{'o': {'l': 1,
  'g': {'enableATCRedesign': 'false',
   'lazyLoaderType': False,
   'largerImageSize': 'FIXED_15',
   'signedIn': False,
   'epInfo': {'featureFlagDetails': {'dwebDeliSSE': {'value': {'NORI_ENABLED': 'true'},
      'dropTag': 'server'},
     'enableSMEMerchBTF': {'value': False},
     'dwebAtfStreamingFeature': {'value': {'NORI_ENABLED': 'true',
       'DELI_ENABLED': 'false'}},
     'prefetchResourceFlag': {'value': {'VINODE_ENABLE_PREFETCH_RESOURCES': 'false'}},
     'dwebAtfSplitFeature': {'value': {'VI_TEMPLATE_USE_CASE': 'VI_EVO',
       'ATF_SPLIT': 'DEFAULT'}},
     'enableOTPFeature': {'value': {'enableOTPFlow': 'false',
       'enableGuestOTPAuthFlow': False},
      'dropTag': 'browser'},
     'enableATCRedesign': {'value': {'enableATCLayerV3': 'false'}},
     'BXR_BUNDLED_TEST': {'value': {'ADS_SME_BTF_ENABLED': 'true',
       'NORI_ENABLED': 'false',
       'DELI_ENABLED': 'true'}},
     'lazyLoaderType': {}},
    'qTags': {}},
   'dWebATFSplit': 'DEFAULT',
 

In [7]:
market = json_['o']['g']['marketplaceId'].strip()
country = json_['o']['g']['country'].strip()
language = json_['o']['g']['language'].strip()
locale = json_['o']['g']['locale'].strip()
item_number = json_['o']['g']['itemId'].strip()

market, country, language, locale, item_number

('EBAY-GB', 'GB', 'en', 'en-GB', '355350556206')

In [8]:
input_.loc[a, 'Market'] == 'US'

False

In [9]:
item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-US' or country != 'US' or language != 'en' or locale != 'en-US'

True

In [10]:
input_.loc[a, 'Market'] == 'DE'

False

In [11]:
item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-DE' or country != 'DE' or language != 'de' or locale != 'de-DE'

True

In [12]:
input_.loc[a, 'Market'] == 'UK'

True

In [13]:
item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-GB' or country != 'GB' or language != 'en' or locale != 'en-GB'

False

In [14]:
input_.loc[a, 'Market'] == 'AU'

False

In [15]:
item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-AU' or country != 'AU' or language != 'en' or locale != 'en-AU'

True

In [16]:
list_compatibility = json_['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'] if 'COMPATIBILITY_TABLE' in json_['o']['w'][0][2]['model']['modules'] else []

dict_compatibility = {'market': input_.loc[a, 'Market'],
                      'page': int(list_compatibility[-1]['text'].strip()),
                      'data': {'scopedContext': {'catalogDetails': {'categoryId': list_compatibility[-1]['action']['params']['categoryId'],
                                                                    'itemId': list_compatibility[-1]['action']['params']['itemId'],
                                                                    'marketplaceId': list_compatibility[-1]['action']['params']['marketplaceId'],
                                                                    'sellerName': list_compatibility[-1]['action']['params']['sellerName']}}}} if list_compatibility else ''

dict_compatibility

''

In [25]:
dict_specific = {}
dict_item = json_['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']
for i, (key, value) in zip(range(len(dict_item.items())), dict_item.items()):
    dict_specific[str(i)] = {value['labels'][0]['textSpans'][0]['text'].strip(): value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

dict_specific

{'0': {'Condition': 'For parts or not working'},
 '1': {'Seller notes': '“Item has cosmetic damage on the mirror base where it attaches to the wing mirror. This is purely cosmetic and is perfect to use otherwise.  See picture for reference”'},
 '2': {'Brand': 'Energized Customs'},
 '3': {'Sub Model': 'Includes Chassis Cab'},
 '4': {'Type': 'Standard Mirror Assembly'},
 '5': {'Wing Mirror Part Type': 'Complete Wing Mirror'},
 '6': {'Part Manufacturer': 'Quality Aftermarket'},
 '7': {'Model': 'Relay'},
 '8': {'Manufacturer Part Number': 'Does Not Apply'},
 '9': {'Mirror Adjustment Method': 'Manual Fold'},
 '10': {'Make': 'Citroen'},
 '11': {'Placement on Vehicle': 'Left'},
 '12': {'Colour': 'Black Mirror Cover'},
 '13': {'EAN': '5037692201069'}}

In [ ]:
description_url = json_['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()
# description_url = json_['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip()

description_url

In [ ]:
title = json_['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

title

In [ ]:
subtitle = json_['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in json_['o']['w'][0][2]['model']['modules']['TITLE'] else ''

subtitle

In [ ]:
price = json_['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
# price = json_['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

price

In [ ]:
available, sold = '', ''
list_quantity = json_['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in json_['o']['w'][0][2]['model']['modules'] else []
for quantity in list_quantity:
    if input_.loc[a, 'Market'] in ['US', 'UK', 'AU']:
        if 'available' in quantity['text']:
            available = quantity['text'].strip()
        elif 'sold' in quantity['text']:
            sold = quantity['text'].strip()
    elif input_.loc[a, 'Market'] == 'DE':
        if 'verfügbar' in quantity['text']:
            available = quantity['text'].strip()
        elif 'verkauft' in quantity['text']:
            sold = quantity['text'].strip()

available, sold

In [ ]:
dict_src = {}
list_media = json_['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in json_['o']['w'][0][2]['model']['modules']['PICTURE'] else []
for i, media in zip(range(len(list_media)), list_media):
    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

dict_src

In [ ]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Item Number': input_.loc[a, 'Item Number'],
                         'Title': title,
                         'Subtitle': subtitle,
                         'Price': price,
                         'Available': available,
                         'Sold': sold,
                         'Vehicle W/o Engine': '',
                         'Vehicle W/ Engine': '',
                         'Picture': '',
                         'Url': url,
                         'Description Url': description_url,
                         'Json_Compatibility': json.dumps(dict_compatibility),
                         'Json_Src': json.dumps(dict_src),
                         'Json_Specific': json.dumps(dict_specific)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

In [ ]:
crawler_status = 'ok'

crawler_status

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Market': input_.loc[a, 'Market'],
                             'Page': input_.loc[a, 'Page'],
                             'Page Url': input_.loc[a, 'Page Url'],
                             'No': input_.loc[a, 'No'],
                             'Item Number': input_.loc[a, 'Item Number']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

In [ ]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_item.xlsx', index=False)

output_correct

In [ ]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_item_error.xlsx', index=False)

output_error